In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
sns.set(style="white", context='poster')
from sympy import *
import math
init_printing(use_unicode=True)
from scipy.integrate import solve_ivp

In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
#from model_equations_separate_NC import *
from model_equations_separate_NC_sep_vmax import *


In [4]:

HCOLOR = '#ffc58f'
HCOLOR_light = '#ffd2a9'
PCOLOR = '#99cc99'
PCOLOR_light = '#aad5aa'

DONCOLOR = '#7285b7'
RDONCOLOR = 'black' #'#00334D'
DINCOLOR = '#bbdaff' #'#000066'

DOCCOLOR = '#cc6666'
RDOCCOLOR = 'black' #'#00334D'
DICCOLOR = '#ff9da4' #'#000066'

SHCOLOR = '#ffa98f'
SPCOLOR = '#326232'


ccnpalette = [PCOLOR, HCOLOR, SPCOLOR, SHCOLOR,DONCOLOR, RDONCOLOR, DINCOLOR]
ccnorder = ['Bp', 'Bh', 'Sp', 'Sh', 'DON', 'RDON', 'DIN', ]
cccpalette = [PCOLOR, HCOLOR, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
cccorder = ['Bp[C]', 'Bh[C]', 'Sp[C]', 'Sh[C]', 'DOC', 'RDOC', 'DIC']
ccxpalette = [PCOLOR, HCOLOR]
ccxorder = ['Xp', 'Xh']
hnpalette = [ HCOLOR_light, SPCOLOR, SHCOLOR, DONCOLOR, RDONCOLOR, DINCOLOR]
hnorder = [ 'Bh', 'Sp', 'Sh', 'DON', 'RDON', 'DIN']
hcpalette = [ HCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
hcorder = [ 'Bh[C]', 'Sp[C]', 'Sh[C]', 'DOC', 'RDOC', 'DIC']
pnpalette = [ PCOLOR_light, SPCOLOR, SHCOLOR, DONCOLOR, RDONCOLOR, DINCOLOR]
pnorder = [ 'Bp', 'Sp', 'Sh', 'DON', 'RDON', 'DIN']
pcpalette = [ PCOLOR_light, SPCOLOR, SHCOLOR, DOCCOLOR, RDOCCOLOR, DICCOLOR]
pcorder = [ 'Bp[C]', 'Sp[C]', 'Sh[C]', 'DOC', 'RDOC', 'DIC']
sns.color_palette(ccnpalette + cccpalette, )
# 


In [5]:

 
sns.color_palette([PCOLOR, HCOLOR, PCOLOR_light, HCOLOR_light] )


In [6]:
refdf = pd.read_csv('prelim_bottle.csv')


In [7]:
refdf.head()

,Unnamed: 0.1,Unnamed: 0,day,1A3,cc 1A3,cc mit9312,mit9312,cc Bh[N],Bh[N],cc Bp[N],Bp[N],cc Bh[C],Bh[C],cc Bp[C],Bp[C],t
0,0,0,0.000000,9.297512e+09,4.698967e+09,7.178947e+08,1.032630e+09,10.069216,19.923240,0.307669,0.442556,45.311472,89.654581,2.153684,3.097889,0.000000
1,1,1,1.820833,1.469212e+10,4.648133e+09,1.753684e+09,1.641075e+09,9.960286,31.483118,0.751579,0.703318,44.821287,141.674030,5.261053,4.923225,157319.999971
2,2,2,4.832639,6.030062e+09,2.996029e+09,9.823158e+09,2.853167e+09,6.420061,12.921562,4.209925,1.222786,28.890276,58.147028,29.469474,8.559501,417540.000010
3,3,3,6.853472,4.987215e+09,2.290707e+09,2.812842e+10,1.576008e+10,4.908658,10.686889,12.055038,6.754319,22.088959,48.091001,84.385263,47.280230,592139.999981
4,4,4,8.779167,2.867312e+09,1.598094e+09,6.262316e+10,4.113340e+10,3.424487,6.144239,26.838496,17.628599,15.410189,27.649077,187.869474,123.400192,758520.000029


In [8]:
dir(DISABLE_MECHANISMS)

['COMPETITION',
 'DETOXIFICATION',
 'H_EXUDATION',
 'H_OVERFLOW',
 'H_RECYCLING',
 'H_SIGNAL',
 'MIXOTROPHY',
 'P_EXUDATION',
 'P_OVERFLOW',
 'P_RECYCLING',
 'P_SIGNAL',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

In [9]:
def run_with_mechanisms_disabled(m):
    print(m)
    new_params = disable_mechanism(m, param_vals)
    #prelim_bottle.csv
    err = generate_json_and_run(
        new_params, os.path.join('prepare data', 'refdf_10cc_12.csv.gz'), 'mechanism_json', 'mechanism_out', 
        f"dis_{ str(m).replace('DISABLE_MECHANISMS.', '').replace('|','-') }", timeout=5*60)

    print (m, err)

In [10]:
os.makedirs("mechanism_out", exist_ok=True)
os.makedirs("mechanism_json", exist_ok=True)


In [11]:
list(DISABLE_MECHANISMS)

[<DISABLE_MECHANISMS.COMPETITION: 1>,
 <DISABLE_MECHANISMS.P_RECYCLING: 2>,
 <DISABLE_MECHANISMS.H_RECYCLING: 4>,
 <DISABLE_MECHANISMS.P_EXUDATION: 8>,
 <DISABLE_MECHANISMS.H_EXUDATION: 16>,
 <DISABLE_MECHANISMS.P_OVERFLOW: 32>,
 <DISABLE_MECHANISMS.H_OVERFLOW: 64>,
 <DISABLE_MECHANISMS.MIXOTROPHY: 128>,
 <DISABLE_MECHANISMS.DETOXIFICATION: 256>,
 <DISABLE_MECHANISMS.P_SIGNAL: 512>,
 <DISABLE_MECHANISMS.H_SIGNAL: 1024>]

In [19]:
alldfs = [run_with_mechanisms_disabled(m) for  m in [DISABLE_MECHANISMS(0)] + list(DISABLE_MECHANISMS)]

DISABLE_MECHANISMS.0
stdout: simulation time 76.3125

MSE: 1447866.0362166672

stderr: 
DISABLE_MECHANISMS.0 1447866.0362166672
DISABLE_MECHANISMS.COMPETITION
stdout: simulation time 61.84375

MSE: 1495472.4606348095

stderr: 
DISABLE_MECHANISMS.COMPETITION 1495472.4606348095
DISABLE_MECHANISMS.P_RECYCLING
stdout: simulation time 74.0

MSE: 1498176.5534550054

stderr: 
DISABLE_MECHANISMS.P_RECYCLING 1498176.5534550054
DISABLE_MECHANISMS.H_RECYCLING
stdout: simulation time 76.640625

MSE: 1511861.0575495372

stderr: 
DISABLE_MECHANISMS.H_RECYCLING 1511861.0575495372
DISABLE_MECHANISMS.P_EXUDATION
stdout: simulation time 77.5

MSE: 1451470.0139656535

stderr: 
DISABLE_MECHANISMS.P_EXUDATION 1451470.0139656535
DISABLE_MECHANISMS.H_EXUDATION
stdout: simulation time 61.046875

MSE: 1396973.377021325

stderr: 
DISABLE_MECHANISMS.H_EXUDATION 1396973.377021325
DISABLE_MECHANISMS.P_OVERFLOW
stdout: simulation time 53.984375

MSE: 1511546.5691876495

stderr: 
DISABLE_MECHANISMS.P_OVERFLOW 151154

KeyboardInterrupt: 

In [13]:
#alldfs = [run_with_mechanisms_disabled(m) for  m in [DISABLE_MECHANISMS(0)] + list(DISABLE_MECHANISMS)]

from itertools import combinations
comb2 = [i | j for i,j in (combinations(DISABLE_MECHANISMS, 2))]
alldfs2 = [run_with_mechanisms_disabled(m) for  m in comb2]


DISABLE_MECHANISMS.P_RECYCLING|COMPETITION
stdout: simulation time 77.5

MSE: 3501.3385554655197

stderr: 
DISABLE_MECHANISMS.P_RECYCLING|COMPETITION 3501.3385554655197
DISABLE_MECHANISMS.H_RECYCLING|COMPETITION
stdout: simulation time 101.640625

MSE: 2747.312430912841

stderr: 
DISABLE_MECHANISMS.H_RECYCLING|COMPETITION 2747.312430912841
DISABLE_MECHANISMS.P_EXUDATION|COMPETITION
TimeoutExpired 120
stdout: 
stderr: 
DISABLE_MECHANISMS.P_EXUDATION|COMPETITION 1e+50
DISABLE_MECHANISMS.H_EXUDATION|COMPETITION
stdout: simulation time 73.4375

MSE: 1680.5115807746595

stderr: 
DISABLE_MECHANISMS.H_EXUDATION|COMPETITION 1680.5115807746595
DISABLE_MECHANISMS.P_OVERFLOW|COMPETITION
stdout: simulation time 66.421875

MSE: 1933.6504613950026

stderr: 
DISABLE_MECHANISMS.P_OVERFLOW|COMPETITION 1933.6504613950026
DISABLE_MECHANISMS.H_OVERFLOW|COMPETITION
stdout: simulation time 71.125

MSE: 1932.2604404433187

stderr: 
DISABLE_MECHANISMS.H_OVERFLOW|COMPETITION 1932.2604404433187
DISABLE_MECHANIS

KeyboardInterrupt: 

In [ ]:
[DISABLE_MECHANISMS(0)] + list(DISABLE_MECHANISMS)

In [ ]:
from itertools import combinations
comb2 = [i | j for i,j in (combinations(DISABLE_MECHANISMS, 2))]
comb2

In [ ]:
dislist = [DISABLE_MECHANISMS(0)] + list(DISABLE_MECHANISMS) + comb2

In [ ]:
str(dislist[20])

In [ ]:
l = [DISABLE_MECHANISMS['P_SIGNAL'] , DISABLE_MECHANISMS['COMPETITION']]

In [ ]:
l

In [ ]:
len(dislist)

In [ ]:
mdf = pd.concat(alldfs + alldfs2)

In [ ]:
mdf.columns

In [ ]:
mdf.loc[mdf.disable_mechanism == '0', 'disable_mechanism'] = 'default'

In [ ]:
mdf['disable_mechanism'] = mdf['disable_mechanism'].str.lower()

In [ ]:
mdf.disable_mechanism.unique()

In [ ]:
mdf.disable_mechanism.nunique()

In [ ]:
g1 = sns.relplot(data=mdf.loc[mdf.variable.isin(ccnorder)], 
            x='day', y='value', hue='variable',
            kind='line', 
            lw=10,
            palette=ccnpalette, hue_order=ccnorder,
            col='disable_mechanism', col_wrap=8,
           ).set(ylabel='umol N/l').set_titles('{col_name}')

for ax in g1.axes_dict.values():
    sns.scatterplot(data=refdf, x='day', y='cc Bp[N]', ax=ax, s=300, legend=False, color=PCOLOR)
    sns.scatterplot(data=refdf, x='day', y='cc Bh[N]', ax=ax, s=300, legend=False, color=HCOLOR)

g2 = sns.relplot(data=mdf.loc[mdf.variable.isin(cccorder)], 
            x='day', y='value', hue='variable',
            kind='line', 
            lw=10,
            palette=cccpalette, hue_order=cccorder,
            col='disable_mechanism', col_wrap=8,
            
           ).set(ylabel='umol C/l').set_titles('{col_name}')
for ax in g2.axes_dict.values():
    sns.scatterplot(data=refdf, x='day', y='cc Bp[C]', ax=ax, s=300, legend=False, color=PCOLOR)
    sns.scatterplot(data=refdf, x='day', y='cc Bh[C]', ax=ax, s=300, legend=False, color=HCOLOR)

# g = sns.relplot(data=mdf.loc[mdf.variable.isin(['Xp', 'Xh'])], 
#             x='day', y='value', hue='variable',
#             kind='line', 
#             lw=2,
#             palette=ccxpalette, hue_order=ccxorder,
#             col='disable_mechanism', col_wrap=4,
                
#            ).set(ylabel='cells/l').set_titles('{col_name}')


In [ ]:
mdf[['mechanism1', 'mechanism2']] = mdf.disable_mechanism.str.split('|', expand=True)
mdf.loc[mdf['mechanism2'].isna(), 'mechanism2'] = mdf.loc[mdf['mechanism2'].isna(), 'mechanism1']


In [ ]:
g = sns.relplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
            x='day', y='value', 
            kind='line', 
            #lw=2,
            #palette=['black'] + sns.color_palette('Dark2', 6),
            color=PCOLOR, lw=20, alpha=1,
            #hue='disable_mechanism', 
            col='disable_mechanism', col_wrap=8,
            aspect=1.4, legend=False,
           ).set(ylabel='PRO umol N/L').set_titles('{col_name}')

#g.map(sns.lineplot, 'day', 'value', color=PCOLOR, lw=10, alpha=1)
for ax in g.axes_dict.values():
    sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
            x='day', y='value', 
            palette='Greys', lw=2, alpha=0.5,
            hue='disable_mechanism', 
            legend=False, ax=ax, zorder=0,
           )
    sns.scatterplot(data=refdf, x='day', y='cc Bp[N]', s=500, legend=False, color=PCOLOR, label='cc mit9312', ax=ax)


In [ ]:
def pp(**kwargs):
    
    # get a reference to the currently active axes
    ax = plt.gca()    
#     sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
#             x='day', y='value', 
#             palette='Greys', lw=2, alpha=0.5,
#             style='disable_mechanism', 
#             legend=False, ax=ax, zorder=0,
#            )
    sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp']) & mdf.mechanism1.isin(['default'])],
            x='day', y='value', 
            color='black', lw=3, alpha=1, ls='--',
            legend=False, ax=ax, #zorder=1,
           )
    sns.scatterplot(data=refdf, x='day', y='cc Bp[N]', s=300, legend=False, color=PCOLOR_light, label='cc mit9312', ax=ax)

g = sns.relplot(data=mdf.loc[mdf.variable.isin(['Bp']) & ~mdf.mechanism1.isin(['default'])], 
            x='day', y='value', 
            kind='line', 
            color=PCOLOR, lw=15, alpha=1,
            col='mechanism1', row='mechanism2',
            aspect=1, legend=False, facet_kws=dict(margin_titles=True),
           )

g.map(pp)
g.set(ylabel='PRO umol N/L')
g.set_titles(col_template='{col_name}', row_template='{row_name}')

# #g.map(sns.lineplot, 'day', 'value', color=PCOLOR, lw=10, alpha=1)
# for ax in g.axes_dict.values():
#     sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
#             x='day', y='value', 
#             palette='Greys', lw=2, alpha=0.5,
#             style='disable_mechanism', 
#             legend=False, ax=ax, zorder=0,
#            )
#     sns.scatterplot(data=refdf, x='day', y='cc Bp[N]', s=500, legend=False, color=PCOLOR, label='cc mit9312', ax=ax)


In [ ]:
def pp(**kwargs):
    
    # get a reference to the currently active axes
    ax = plt.gca()    
#     sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
#             x='day', y='value', 
#             palette='Greys', lw=2, alpha=0.5,
#             style='disable_mechanism', 
#             legend=False, ax=ax, zorder=0,
#            )
    sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp']) & mdf.mechanism1.isin(['default'])],
            x='day', y='value', 
            color='black', lw=3, alpha=1, ls='--',
            legend=False, ax=ax, #zorder=1,
           )
    sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bh']) & mdf.mechanism1.isin(['default'])],
            x='day', y='value', 
            color='black', lw=3, alpha=1, ls='--',
            legend=False, ax=ax, #zorder=1,
           )
    sns.scatterplot(data=refdf, x='day', y='cc Bp[N]', ax=ax, s=300, legend=False, color=PCOLOR)
    sns.scatterplot(data=refdf, x='day', y='cc Bh[N]', ax=ax, s=300, legend=False, color=HCOLOR)

g = sns.relplot(data=mdf.loc[mdf.variable.isin(ccnorder)& ~mdf.mechanism1.isin(['default'])], 
            x='day', y='value', hue='variable',
            kind='line', 
            lw=10,
            palette=ccnpalette, hue_order=ccnorder,
            col='mechanism1', row='mechanism2',
            aspect=1, legend=False, facet_kws=dict(margin_titles=True),
           )

g.map(pp)
g.set(ylabel='umol N/L')
g.set_titles(col_template='{col_name}', row_template='{row_name}')

# #g.map(sns.lineplot, 'day', 'value', color=PCOLOR, lw=10, alpha=1)
# for ax in g.axes_dict.values():
#     sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
#             x='day', y='value', 
#             palette='Greys', lw=2, alpha=0.5,
#             style='disable_mechanism', 
#             legend=False, ax=ax, zorder=0,
#            )
#     sns.scatterplot(data=refdf, x='day', y='cc Bp[N]', s=500, legend=False, color=PCOLOR, label='cc mit9312', ax=ax)


In [ ]:
def pp(**kwargs):
    
    # get a reference to the currently active axes
    ax = plt.gca()    
#     sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
#             x='day', y='value', 
#             palette='Greys', lw=2, alpha=0.5,
#             style='disable_mechanism', 
#             legend=False, ax=ax, zorder=0,
#            )

    for c,i in zip(cccpalette, cccorder):

        sns.lineplot(data=mdf.loc[mdf.variable.isin([i]) & mdf.mechanism1.isin(['default'])],
                x='day', y='value', 
                color=c, lw=2, alpha=1, ls='--',
                legend=False, ax=ax, #zorder=1,
               )
    sns.scatterplot(data=refdf, x='day', y='cc Bp[C]', ax=ax, s=300, legend=False, color=PCOLOR)
    sns.scatterplot(data=refdf, x='day', y='cc Bh[C]', ax=ax, s=300, legend=False, color=HCOLOR)

g = sns.relplot(data=mdf.loc[mdf.variable.isin(cccorder)& ~mdf.mechanism1.isin(['default'])], 
            x='day', y='value', hue='variable',
            kind='line', 
            lw=10,
            palette=cccpalette, hue_order=cccorder,
            col='mechanism1', row='mechanism2',
            aspect=1, legend=False, facet_kws=dict(margin_titles=True),
           )

g.map(pp)
g.set(ylabel='umol C/L')
g.set_titles(col_template='{col_name}', row_template='{row_name}')

# #g.map(sns.lineplot, 'day', 'value', color=PCOLOR, lw=10, alpha=1)
# for ax in g.axes_dict.values():
#     sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
#             x='day', y='value', 
#             palette='Greys', lw=2, alpha=0.5,
#             style='disable_mechanism', 
#             legend=False, ax=ax, zorder=0,
#            )
#     sns.scatterplot(data=refdf, x='day', y='cc Bp[N]', s=500, legend=False, color=PCOLOR, label='cc mit9312', ax=ax)


In [ ]:
d = mdf.loc[mdf.variable.isin(['Bp']) & ~mdf.mechanism1.isin(['default'])]

for m in d.mechanism1.unique():
    g = sns.relplot(
        data=mdf.loc[mdf.variable.isin(['Bp']) & (mdf.mechanism1.isin([m]) | mdf.mechanism2.isin([m]))], 
        x='day', y='value', 
        kind='line', style='disable_mechanism', #dashes=False, markers=True,
        color=PCOLOR, lw=3, alpha=1,
        aspect=1, legend=False,
               )
    

    sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp']) & mdf.mechanism1.isin(['default'])],
            x='day', y='value', 
            color='black', lw=3, alpha=1, ls='--',
            legend=False, #ax=ax, #zorder=1,
           )
    sns.scatterplot(data=refdf, x='day', y='cc Bp[N]', s=100, legend=False, color=PCOLOR_light, label='cc mit9312', #ax=ax
                   )

    g.set(ylabel='PRO umol N/L', title=m, ylim=(0,120))
    
#g.set_titles(col_template='{col_name}', row_template='{row_name}')


# #g.map(sns.lineplot, 'day', 'value', color=PCOLOR, lw=10, alpha=1)
# for ax in g.axes_dict.values():
#     sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
#             x='day', y='value', 
#             palette='Greys', lw=2, alpha=0.5,
#             style='disable_mechanism', 
#             legend=False, ax=ax, zorder=0,
#            )
#     sns.scatterplot(data=refdf, x='day', y='cc Bp[N]', s=500, legend=False, color=PCOLOR, label='cc mit9312', ax=ax)


In [ ]:
d = mdf.loc[mdf.variable.isin(['Bh']) & ~mdf.mechanism1.isin(['default'])]

for m in d.mechanism1.unique():
    g = sns.relplot(
        data=mdf.loc[mdf.variable.isin(['Bh']) & (mdf.mechanism1.isin([m]) | mdf.mechanism2.isin([m]))], 
        x='day', y='value', 
        kind='line', style='disable_mechanism', #dashes=False, markers=True,
        color=HCOLOR, lw=3, alpha=1,
        aspect=1, legend=False,
               )
    

    sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bh']) & mdf.mechanism1.isin(['default'])],
            x='day', y='value', 
            color='black', lw=3, alpha=1, ls='--',
            legend=False, #ax=ax, #zorder=1,
           )
    sns.scatterplot(data=refdf, x='day', y='cc Bh[N]', s=100, legend=False, color=HCOLOR_light, label='cc mit9312', #ax=ax
                   )

    g.set(ylabel='HET umol N/L', title=m, ylim=(0,120))
    
#g.set_titles(col_template='{col_name}', row_template='{row_name}')


# #g.map(sns.lineplot, 'day', 'value', color=PCOLOR, lw=10, alpha=1)
# for ax in g.axes_dict.values():
#     sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
#             x='day', y='value', 
#             palette='Greys', lw=2, alpha=0.5,
#             style='disable_mechanism', 
#             legend=False, ax=ax, zorder=0,
#            )
#     sns.scatterplot(data=refdf, x='day', y='cc Bp[N]', s=500, legend=False, color=PCOLOR, label='cc mit9312', ax=ax)


In [ ]:
def pp(**kwargs):
    
    # get a reference to the currently active axes
    ax = plt.gca()    
#     sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
#             x='day', y='value', 
#             palette='Greys', lw=2, alpha=0.5,
#             style='disable_mechanism', 
#             legend=False, ax=ax, zorder=0,
#            )
    sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bh']) & mdf.mechanism1.isin(['default'])],
            x='day', y='value', 
            color='black', lw=3, alpha=1, ls='--',
            legend=False, ax=ax, #zorder=1,
           )
    sns.scatterplot(data=refdf, x='day', y='cc Bh[N]', s=300, legend=False, color=HCOLOR_light, label='cc mit9312', ax=ax)

g = sns.relplot(data=mdf.loc[mdf.variable.isin(['Bh']) & ~mdf.mechanism1.isin(['default'])], 
            x='day', y='value', 
            kind='line', 
            color=HCOLOR, lw=15, alpha=1,
            col='mechanism1', row='mechanism2',
            aspect=1, legend=False, facet_kws=dict(margin_titles=True),
           )

g.map(pp)
g.set(ylabel='HET umol N/L')
g.set_titles(col_template='{col_name}', row_template='{row_name}')

# #g.map(sns.lineplot, 'day', 'value', color=PCOLOR, lw=10, alpha=1)
# for ax in g.axes_dict.values():
#     sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bp'])], 
#             x='day', y='value', 
#             palette='Greys', lw=2, alpha=0.5,
#             style='disable_mechanism', 
#             legend=False, ax=ax, zorder=0,
#            )
#     sns.scatterplot(data=refdf, x='day', y='cc Bp[N]', s=500, legend=False, color=PCOLOR, label='cc mit9312', ax=ax)


In [ ]:
g = sns.relplot(data=mdf.loc[mdf.variable.isin(['Bh'])], 
            x='day', y='value', 
            kind='line', 
            #lw=2,
            #palette=['black'] + sns.color_palette('Dark2', 6),
            color=HCOLOR, lw=20, alpha=1,
            #hue='disable_mechanism', 
            col='disable_mechanism', col_wrap=8,
            aspect=1.4, legend=False,
           ).set(ylabel='HET umol N/L').set_titles('{col_name}')

#g.map(sns.lineplot, 'day', 'value', color=PCOLOR, lw=10, alpha=1)
for ax in g.axes_dict.values():
    sns.lineplot(data=mdf.loc[mdf.variable.isin(['Bh'])], 
            x='day', y='value', 
            palette='Greys', lw=2, alpha=0.5,
            style='disable_mechanism', 
            legend=False, ax=ax, zorder=0,
           )
    sns.scatterplot(data=refdf, x='day', y='cc Bh[N]', s=500, legend=False, color=HCOLOR, label='cc 1A3', ax=ax)


In [ ]:
g = sns.relplot(data=mdf.loc[mdf.variable.isin(['Xh'])], 
            x='day', y='value', 
            kind='line', 
            #lw=2,
            #palette=['black'] + sns.color_palette('Dark2', 6),
            hue='disable_mechanism', 
            aspect=1.4,
           ).set(ylabel='HET cells/l', ).set_titles('{col_name}')
#sns.scatterplot(data=refdf, x='day', y='cc mit9312', s=100, legend=False, color=PCOLOR, label='cc mit9312')
#sns.scatterplot(data=refdf, x='day', y='mit9312', s=100, legend=False, color=PCOLOR_light, label='mit9312', marker='P')
sns.scatterplot(data=refdf, x='day', y='cc 1A3[]',  s=100, legend=False,color=HCOLOR, label='cc 1A3')


In [ ]:
g = sns.relplot(data=mdf.loc[mdf.variable.isin(['Xh'])], 
            x='day', y='value', 
            kind='line', 
            #lw=2,
            #palette=['black'] + sns.color_palette('Dark2', 6),
            hue='disable_mechanism', 
            aspect=1.4,
           ).set(ylabel='HET cells/l', yscale='log' ).set_titles('{col_name}')
#sns.scatterplot(data=refdf, x='day', y='cc mit9312', s=100, legend=False, color=PCOLOR, label='cc mit9312')
#sns.scatterplot(data=refdf, x='day', y='mit9312', s=100, legend=False, color=PCOLOR_light, label='mit9312', marker='P')
sns.scatterplot(data=refdf, x='day', y='cc 1A3',  s=100, legend=False,color=HCOLOR, label='cc 1A3')


In [ ]:
g = sns.relplot(data=mdf.loc[mdf.variable.isin(['Xp'])], 
            x='day', y='value', 
            kind='line', 
            #lw=2,
            #palette=['black'] + sns.color_palette('Dark2', 6),
            hue='disable_mechanism', 
            aspect=1.4,
           ).set(ylabel='PRO cells/l', yscale='log').set_titles('{col_name}')
sns.scatterplot(data=refdf, x='day', y='cc mit9312', s=100, legend=False, color=PCOLOR, label='cc mit9312')
#sns.scatterplot(data=refdf, x='day', y='mit9312', s=100, legend=False, color=PCOLOR_light, label='mit9312', marker='P')


In [ ]:
len(param_vals)

In [ ]:
mdf.to_csv('disable_mechanism_res.csv.gz')